In [5]:
import os
import json
from tqdm import tqdm
import numpy as np
import pandas as pd

import ray
from ray.rllib.offline.json_writer import JsonWriter
from ray.rllib.offline.json_reader import JsonReader
from ray.rllib.evaluation.sample_batch_builder import SampleBatchBuilder

import gzip
import pickle

from dopamine.replay_memory import circular_replay_buffer
from dopamine.replay_memory import prioritized_replay_buffer

import gin
import tensorflow.compat.v1 as tf

In [6]:
from batch_rl.batch_rl.baselines.replay_memory import logged_prioritized_replay_buffer

replay = logged_prioritized_replay_buffer.WrappedLoggedPrioritizedReplayBuffer(
        log_dir="AC_datasets/v5_scoremax_test_dataset_cql/replay_logs",
        observation_shape=(80,),
        stack_size=1,
        use_staging=False,
        update_horizon=1,
        gamma=0.99,
        observation_dtype=np.float32
)

In [7]:
dataset_path = "ac-pulse-rl/data/processed/v5_dataset/scores_max/test_dataset/"
dataset = []

for path in os.listdir(dataset_path):
    if not os.path.isfile(os.path.join(dataset_path, path)):
        dataset.append(path)
dataset = sorted(dataset)
len(dataset)

81

In [8]:
for data in tqdm(dataset):
    dataset_files = []
    for files in os.listdir(os.path.join(dataset_path, data)):
        if os.path.isfile(os.path.join(dataset_path, data, files)):
            dataset_files.append(os.path.join(data, files))
    dataset_files = sorted(dataset_files)

    for dataset_file in dataset_files:
        reader = JsonReader(os.path.join(dataset_path, dataset_file))
        
        with open(os.path.join(dataset_path, dataset_file), "r") as f:
            sb=f.readlines()

        for _ in range(len(sb)):
            batch = reader.next()

            for i in range(len(batch["eps_id"])):
                replay.add(batch["obs"][i], batch["actions"][i], batch["rewards"][i], batch["dones"][i], 1)
                
replay.memory.log_final_buffer()

100%|██████████| 81/81 [00:04<00:00, 17.98it/s]


INFO:tensorflow:Replay buffer logged to ckpt 0 in AC_datasets/v5_scoremax_test_dataset_cql/replay_logs


INFO:tensorflow:Replay buffer logged to ckpt 0 in AC_datasets/v5_scoremax_test_dataset_cql/replay_logs
